In [ ]:
# Uncomment to install the required package
# !pip install transformers


In [ ]:
# Load the welfare scheme write-up from a text file
file_path = "welfare_scheme.txt"  # Ensure the file is in the same directory

with open(file_path, "r") as file:
    write_up = file.read()

print("Welfare scheme write-up loaded successfully.")


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

# Load the Pegasus tokenizer and model
model_name = "google/pegasus-xsum"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)


In [ ]:
def get_summary_with_pegasus(write_up: str, prompt_section: str) -> str:
    """
    Summarizes the welfare scheme write-up based on the specified prompt section using Pegasus.

    Args:
    - write_up (str): Full write-up of the welfare scheme.
    - prompt_section (str): The section prompt, e.g., "Beneficiary and Problem Statement".

    Returns:
    - str: The generated summary for the specified section.
    """
    # Customize the prompt for the section
    prompt = f"{prompt_section}: {write_up}"

    # Tokenize the input
    inputs = tokenizer(prompt, max_length=1024, truncation=True, return_tensors="pt")

    # Generate summary
    summary_ids = model.generate(inputs.input_ids, max_length=60, num_beams=5, length_penalty=1.2, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    return summary

def summarize_scheme_with_pegasus(write_up: str) -> dict:
    """
    Generates summaries for each of the three sections using Pegasus:
    'Beneficiary and Problem Statement', 'Application Process and Benefits', 'Outcome and Impact'.
    """
    sections = [
        "Beneficiary and Problem Statement",
        "Application Process and Benefits",
        "Outcome and Impact"
    ]
    
    summaries = {}
    for section in sections:
        summaries[section] = get_summary_with_pegasus(write_up, section)
    
    return summaries


In [ ]:
# Generate summaries
summaries = summarize_scheme_with_pegasus(write_up)

# Display the summaries
for section, summary in summaries.items():
    print(f"{section}:\n{summary}\n")